In [1]:
import os
from tracklet_claude import *
home = os.path.expanduser("~")

data_dir = os.path.join(home,"buni", "output-data","av2", "bbox-estimation")

scene_idx = 0
scene_id = os.listdir(data_dir)[scene_idx]

scene_path = os.path.join(data_dir, scene_id)

print(scene_id)
print(scene_path)


print(os.path.join(scene_path, os.listdir(scene_path)[0]))


ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd
C:\Users\Ravi\buni\output-data\av2\bbox-estimation\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd
C:\Users\Ravi\buni\output-data\av2\bbox-estimation\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\315968335160099000.feather


In [ ]:
test_df = pd.read_feather(os.path.join(scene_path, os.listdir(scene_path)[0]))
frame_detections_df = [(pd.read_feather(os.path.join(scene_path, frame_id)), frame_id.split(".")[0]) for frame_id in os.listdir(scene_path)]
# Convert to frame-level BoundingBox lists
# frame_detections is a list of lists. Main list contains a list of detections for each frame.
# Each sublist corresponds to a single frame and is a list of bboxes(detections) for that frame
# each elem in frame_detections_df => (df, frame_id)  
frame_detections = [convert_to_bounding_boxes(df_frameid[0], df_frameid[1]) for df_frameid in frame_detections_df] 

In [3]:
frame0 = frame_detections[0]

In [4]:
tracker0 = OnlineTracker()
track_results0 = tracker0.update(frame0)

In [9]:
two_tracker = BidirectionalTracker()
# forward_results, backward_results = two_tracker.track_sequence(frame_detections)
# save_tracking_results(forward_results, backward_results,os.path.join(os.getcwd(),"saved_tracklets"))

In [7]:
load_path = os.path.join(os.getcwd(),"saved_tracklets")

with open(os.path.join(load_path, "forward.pkl"), "rb") as f:
    forward_results = pickle.load(f)

with open(os.path.join(load_path, "backward.pkl"), "rb") as f:
    backward_results = pickle.load(f)


In [10]:
frame_with_temp_scores =two_tracker.compute_temporal_scores(forward_results, backward_results)

In [ ]:
from enum import Enum

class Corner(Enum):
    REAR_LEFT = 0
    REAR_RIGHT = 1
    FRONT_RIGHT = 2
    FRONT_LEFT = 3

def calculate_centroid_from_corner(corner_coords: np.ndarray, dimensions: np.ndarray, angle: float, corner_type: Corner):
    """
    Calculates the centroid of a bounding box given a corner, its dimensions, and orientation.
    
    Args:
        corner_coords: np.ndarray
            The [x, y] coordinates of the anchor corner.
        dimensions: np.ndarray
            The [length, width] of the bounding box.
        angle: float
            The orientation angle (in radians).
        corner_type: Corner (Enum)
            The type of corner (REAR_LEFT, REAR_RIGHT, FRONT_RIGHT, FRONT_LEFT).
    
    Returns:
        np.ndarray: [x, y] coordinates of the centroid. Shape (2,).
    """
    l, w = dimensions  # Length and width
    x_corner, y_corner = corner_coords  # Unpack corner coordinates
    
    # Determine unrotated offset to centroid based on corner type
    if corner_type == Corner.REAR_LEFT:
        offset = np.array([w / 2, l / 2])
    elif corner_type == Corner.REAR_RIGHT:
        offset = np.array([-w / 2, l / 2])
    elif corner_type == Corner.FRONT_RIGHT:
        offset = np.array([-w / 2, -l / 2])
    elif corner_type == Corner.FRONT_LEFT:
        offset = np.array([w / 2, -l / 2])
    else:
        raise ValueError("Invalid corner type provided.")
    
    # Rotate the offset based on the angle
    rotation_matrix = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)]
    ])
    rotated_offset = rotation_matrix @ offset
    
    # Calculate the centroid
    centroid = np.array([x_corner, y_corner]) + rotated_offset
    return centroid
    

def find_closest_corner(corners: np.ndarray, ego_vehicle: np.ndarray = None):
    """
    Finds the closest corner to the ego-vehicle.
    """
    # find the closest corner and its index in corners array
    if ego_vehicle is None:
        ego_vehicle = np.array([0, 0])
    min_dist = float("inf")
    closest_corner = None
    closest_corner_idx = None
    for i, corner in enumerate(corners):
        dist = np.linalg.norm(corner - ego_vehicle)
        if dist < min_dist:
            min_dist = dist
            closest_corner = corner
            closest_corner_idx = i
    
    return closest_corner, closest_corner_idx

def refine_bbox(bbox_tracklet_dict: Dict, r_percentile=95):
    """
    Refines a bbox by finding the closet corner to ego-vehicle 
    and updating the lenght and width of the box.
    
    Args:
        bbox_tracklet_dict: Dict
            A dictionary containing a bbox and a tracklet.
            {"bbox": BoundingBox, forward_tracklet: Tracklet, backward_tracklet: Tracklet}
        r_percentile: int
            
    Returns:
        BoundingBox: A refined bounding box.
    """

    # find corner closest to ego-vehicle
    bbox = bbox_tracklet_dict["bbox"]
    corners = get_bbox_corners((bbox.x, bbox.y, bbox.l, bbox.w, bbox.theta))
    
    closest_corner, closest_corner_idx = find_closest_corner(corners)
    
    corner_type = Corner(closest_corner_idx)
    
    forward_dimensions_lst = [(bbox.l, bbox.w) for bbox in bbox_tracklet_dict["forward_tracklet"].boxes]
    backward_dimesnions_lst = [(bbox.l, bbox.w) for bbox in bbox_tracklet_dict["backward_tracklet"].boxes]
    
    # find 95th percentile of the dimensions
    new_dimensions = np.percentile(forward_dimensions_lst, r_percentile, axis=0)
    
    new_centroid = calculate_centroid_from_corner(closest_corner, new_dimensions, bbox.theta, corner_type)
    
    new_bbox = BoundingBox(new_centroid[0], new_centroid[1], new_dimensions[0], new_dimensions[1], bbox.theta, frame_id=bbox.frame_id)
    
    return new_bbox

def refine_tracklets(bbox_trackelts_lst_lst_dic: List[List[Dict]]) -> List[List[BoundingBox]]:
    """
    Refines tracklets by filtering out bboxes with low temporal scores and refining the bboxes of an entire scene
    
    Args:
        bbox_trackelts_lst_lst_dic: List[List[Dict]]
            A list of list of dictionaries containing bboxes and tracklets.
            Each sublist corresponds to a single frame and is a list of bboxes(detections) for that frame.
            Each elem in frame_detectiosn_df => (df, frame_id)  
            Each dictionary in the sublist contains a bbox and its corresponding tracklet.

    Returns:
        List[List[BoundingBox]]: A list of list of refined bounding boxes.
        
    List[BoundingBox] represents a single frame and
    List[List[BoundingBox]] represents a single scene.
    """
    filtered_lst_lst_dic = []
    q = 6
    for frame in bbox_trackelts_lst_lst_dic:
        filtered_frame = []
        for bbox_tracklet_dict in frame:
            # print(f"type of bbox_tracklet_tuple: {type(bbox_tracklet_tuple)}")
            if bbox_tracklet_dict["bbox"].temporal_score > q:
                # print(f"temporal score is {bbox_tracklet_dict['bbox'].temporal_score}")
                filtered_frame.append(bbox_tracklet_dict)
            
        filtered_lst_lst_dic.append(filtered_frame)
        
    filtered_scenes_list = []
    for frame in bbox_trackelts_lst_lst_dic:
        filtered_frames_lst = []
        for bbox_tracklet_dict in frame:
            new_bbox = refine_bbox(bbox_tracklet_dict)
            filtered_frames_lst.append(new_bbox)
        filtered_scenes_list.append(filtered_frames_lst)
    
    return filtered_scenes_list
        
        

In [31]:
refined_bboxes = refine_tracklets(frame_with_temp_scores)

In [32]:
def save_refined_scene(refined_scene: List[List[BoundingBox]], save_path: str):
    """
    Saves the refined scene in separate .feather files for each frame.
    Each frame is saved as save_path/frame_id.feather.

    Args:
        refined_scene: List[List[BoundingBox]]
            A list of lists of refined bounding boxes for each frame.
        save_path: str
            The path to save the refined scene.

    Returns:
        List[str]: A list of file paths for the saved .feather files.
    """
    saved_paths = []

    for frame_idx, frame in enumerate(refined_scene):
        # Select only x, y, l, w, theta for each bounding box
        frame_df = pd.DataFrame([{
            "box_center_x": bbox.x,
            "box_center_y": bbox.y,
            "box_length": bbox.l,
            "box_widht": bbox.w,
            "ry": bbox.theta
        } for bbox in frame])
        
        # Save each frame as a separate .feather file
        frame_id = frame[0].frame_id if len(frame) > 0 else frame_idx  # Use frame_id if available, otherwise use frame index
        feather_path = os.path.join(save_path, f"{frame_id}.feather")
        frame_df.to_feather(feather_path)
        
        saved_paths.append(feather_path)

    return saved_paths


    

In [35]:
scene_id

'ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd'

In [34]:
base_save_path = os.path.join(os.path.expanduser("~"), "buni", "output-data", "av2", "refined_bboxes")
save_path = os.path.join(base_save_path, scene_id)
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
save_refined_scene(refined_bboxes, save_path)



['C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335160099000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335259632000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335359828000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335460024000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335560221000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335659754000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335759950000.feather',
 'C:\\Users\\Ravi\\buni\\output-data\\av2\\refined_bboxes\\ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd\\315968335860146000.fe